In [1]:
%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


# Setting up image pipeline


In [2]:
# download dataset
IMAGES_DIR = '/content/data'
try:
  os.makedirs(IMAGES_DIR)
except:
  print('Already exists')

tf.keras.utils.get_file("MelanomaDetection.zip","https://lp-prod-resources.s3.amazonaws.com/278/45149/2021-02-19-19-47-43/MelanomaDetection.zip", extract=True)

Already exists
6758400/6744318 [==============================] - 2s 0us/step


'/root/.keras/datasets/MelanomaDetection.zip'

In [47]:
import matplotlib.pyplot as plt
import pandas as pd
from PIL import Image
import random
from random import shuffle
import numpy as np
import matplotlib.pyplot as plt
import copy

import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision.io import read_image
import torchvision.utils as vutils
import torchvision.transforms as T
import torch.nn.functional as F

import cv2
from keras.preprocessing.image import  img_to_array
from glob import glob
import os
from tqdm.notebook import tqdm


In [4]:
# Decide which device we want to run on
device = torch.device("cuda:0" if (torch.cuda.is_available() ) else "cpu")

In [5]:
!mv ~/.keras/datasets/MelanomaDetection/ ./data 

In [6]:
def is_image(filename):
    return any(filename.endswith(extension) for extension in [".jpg", ".jpeg", ".png"])

In [7]:
class UnlabeledImagesDataSet(torch.utils.data.Dataset):
  def __init__(self, dir_path, transform=None):
      super(UnlabeledImagesDataSet, self).__init__()
      self.path = dir_path
      self.transform = transform     
      self.files = glob(os.path.join(self.path,'*.jpg'), recursive=True)  
      self.files.extend(glob(os.path.join(self.path,'*.jpeg'), recursive=True))
      self.files.extend(glob(os.path.join(self.path,'*.png'), recursive=True))
  def __len__(self):
    return len(self.files)

  def __getitem__(self,i):
    img_path =self.files[i]
    #print(f'Getting {img_path}')
    image = Image.open(img_path)   

    if(self.transform):
      image = self.transform(image)

    return image


In [8]:
class LabeledImagesDataSet(torch.utils.data.Dataset):
  def __init__(self, dir_path, transform=None):
      super(LabeledImagesDataSet, self).__init__()
      self.path = dir_path
      self.transform = transform
      self.data = []    
      self.labels = []
      self.files = glob(os.path.join(self.path,'*.jpg'), recursive=True)  
      self.files.extend(glob(os.path.join(self.path,'*.jpeg'), recursive=True))
      self.files.extend(glob(os.path.join(self.path,'*.png'), recursive=True))    
  def __len__(self):
    return len(self.files)

  def __getitem__(self,i):
        img_path =self.files[i]
       
        image = Image.open(img_path)
        if(self.transform):
          image = self.transform(image)
          
        label = 0
        if "_1.jpg" in img_path:
          label = 1        
        return (image,label)

In [9]:
data_folder = "./data/MelanomaDetection"
labeledDS = LabeledImagesDataSet(os.path.join(data_folder, 'labeled'), transform= T.Compose([T.ToTensor()]))

In [10]:
dataloader = DataLoader(labeledDS, batch_size=4,
                        shuffle=True, num_workers=2)

In [11]:
unlabeledDS = UnlabeledImagesDataSet(os.path.join(data_folder,'unlabeled'))

In [12]:
unlabeledDataLoader = DataLoader(unlabeledDS, batch_size=4, shuffle=True, num_workers=2)

In [13]:
from torchvision.transforms import transforms
unlabeledDS = UnlabeledImagesDataSet(os.path.join(data_folder,'unlabeled'), transform= T.Compose([T.ToTensor()]))
unlabeledDataLoader = DataLoader(unlabeledDS, batch_size=16, shuffle=True, num_workers=2)

Transformations and pre-preprocessing


*   Normalisation of pixel intensities
*   Convert colour to grayscale
*   Augmentation can include
 *  Rotation
 *  Skewing 



In [14]:
# split the labelled data into train and test
train_set, val_set = torch.utils.data.random_split(labeledDS, [int(0.7*len(labeledDS)), int(0.3*len(labeledDS))])

In [15]:
len(train_set)

140

In [16]:
n_channels = 3
n_features = 32 #size of image
rand_input = 100
# Learning rate for optimizers
lr = 0.0002

# Beta1 hyperparam for Adam optimizers
beta1 = 0.5


# DCGAN
GAN which employed convolutional and convolutional-transpose layers in both the discriminator and generator.

In [17]:
# custom weights initialization called on netG and netD
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        nn.init.normal_(m.weight.data, 1.0, 0.02)
        nn.init.constant_(m.bias.data, 0)

## Generator

In [18]:
class Generator(nn.Module):
  def __init__(self,len_input,n_features):
    super(Generator,self).__init__()
    self.ngpu = 1
    self.main = nn.Sequential(
        nn.ConvTranspose2d( len_input, n_features * 4, 4, 1, 0, bias=False),
        nn.BatchNorm2d(n_features * 4),
        nn.ReLU(True),
        # state size. (n_features*8) x 4 x 4
        nn.ConvTranspose2d(n_features * 4, n_features * 2, 4, 2, 1, bias=False),
        nn.BatchNorm2d(n_features * 2),
        nn.ReLU(True),
        # state size. (n_features*4) x 8 x 8
        nn.ConvTranspose2d( n_features * 2, n_features , 4, 2, 1, bias=False),
        nn.BatchNorm2d(n_features),
        nn.ReLU(True),
        # state size. (n_features*2) x 16 x 16
        # nn.ConvTranspose2d( n_features * 2, n_features, 4, 2, 1, bias=False),
        # nn.BatchNorm2d(n_features),
        # nn.ReLU(True),
        # state size. (n_features) x 32 x 32
        nn.ConvTranspose2d( n_features, n_channels, 4, 2, 1, bias=False),
        nn.Tanh()        
    )
    # self.conv = nn.Conv2d(3,32,kernel_size=3)
    # self.pool = nn.MaxPool2d(2)
    # self.hidden = nn.Linear()
  def forward(self,x):
    return self.main(x)
    #x0 = self.l1(x)


In [19]:
gen = Generator(rand_input,n_features).to(device)

In [20]:
#init weights
gen = nn.DataParallel(gen,list(range(1)))

In [21]:
gen.apply(weights_init)

DataParallel(
  (module): Generator(
    (main): Sequential(
      (0): ConvTranspose2d(100, 128, kernel_size=(4, 4), stride=(1, 1), bias=False)
      (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
      (3): ConvTranspose2d(128, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
      (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (5): ReLU(inplace=True)
      (6): ConvTranspose2d(64, 32, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
      (7): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (8): ReLU(inplace=True)
      (9): ConvTranspose2d(32, 3, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
      (10): Tanh()
    )
  )
)

## Modified discriminator to classifier

In [22]:
class Classifier(nn.Module):
  def __init__(self,n_features, n_classes):
    super(Classifier,self).__init__()
    self.ngpu = 1
    self.main = nn.Sequential(
        # input is (nc) x 32 x 32
        nn.Conv2d(n_channels, n_features, 4, 2, 1, bias=False),
        nn.LeakyReLU(0.2, inplace=True),
        # state size. (n_features) x 32 x 32
        nn.Conv2d(n_features, n_features * 2, 4, 2, 1, bias=False),
        nn.BatchNorm2d(n_features * 2),
        nn.LeakyReLU(0.2, inplace=True),
        # state size. (n_features*2) x 16 x 16
        nn.Conv2d(n_features * 2, n_features * 4, 4, 2, 1, bias=False),
        nn.BatchNorm2d(n_features * 4),
        nn.LeakyReLU(0.2, inplace=True),
        # state size. (n_features*4) x 8 x 8
        # nn.Conv2d(n_features * 4, n_features * 8, 4, 2, 1, bias=False),
        # nn.BatchNorm2d(n_features * 8),
        # nn.LeakyReLU(0.2, inplace=True),
        # state size. (n_features*8) x 4 x 4
        nn.Conv2d(n_features * 4, n_classes, 4, 1, 0, bias=False),        
    )
  

  def forward(self,input):
    return self.main(input)
    

In [23]:
classifier = Classifier(n_features, 2).to(device)

In [24]:
classifier.apply(weights_init)

Classifier(
  (main): Sequential(
    (0): Conv2d(3, 32, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (1): LeakyReLU(negative_slope=0.2, inplace=True)
    (2): Conv2d(32, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (3): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (4): LeakyReLU(negative_slope=0.2, inplace=True)
    (5): Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (6): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (7): LeakyReLU(negative_slope=0.2, inplace=True)
    (8): Conv2d(128, 2, kernel_size=(4, 4), stride=(1, 1), bias=False)
  )
)

## Supervised learning / representational learning

In [25]:
batch_size = 4
epochs = 30

In [26]:
data_folder = "./data/MelanomaDetection"
labeledDS = LabeledImagesDataSet(os.path.join(data_folder, 'labeled'))

In [27]:
dataloader = DataLoader(labeledDS, batch_size=len(train_set),
                        shuffle=True, num_workers=0)

In [28]:
def set_seed(seed):

    np.random.seed(seed)
    random.seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    torch.manual_seed(seed)

def log_sum_exp(x, axis = 1):
    m = torch.max(x, dim = 1)[0]
    return m + torch.log(torch.sum(torch.exp(x - m.unsqueeze(1)), dim = axis))

In [29]:
import torch.optim as optim
# Initialize BCELoss function
criterion = nn.BCELoss()

# Create batch of latent vectors that we will use to visualize
#  the progression of the generator
fixed_noise = torch.randn(64, rand_input, 1, 1, device=device)

# Establish convention for real and fake labels during training
real_label = 1.
fake_label = 0.





In [30]:
set_seed(10)

In [44]:
criterion = nn.CrossEntropyLoss()

def test(model, device, test_loader, display=False):
    
    # eval() is the mode that "turns off" the non-deterministic layers
    # that may be present in the model (e.g. dropout, batchnorm, etc)
    model.eval()
    
    test_loss = 0
    correct = 0
    
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.type(torch.LongTensor).to(device)
            output = model(data)
            test_loss += criterion(output.squeeze(), target.squeeze()).item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)  
    
    if display:
        print('Test set accuracy: ',(100. * correct / len(test_loader.dataset)))
    
    return test_loss, (100. * correct / len(test_loader.dataset))

In [48]:
num_classes = 2

netG = Generator(rand_input, n_features).to(device)
netG.apply(weights_init)

classifier = Classifier(n_features, num_classes).to(device)
classifier.apply(weights_init)

optimizerG = optim.Adam(netG.parameters(), lr=0.002, betas= (0.5, 0.999))
optimizerC = optim.Adam(classifier.parameters(), lr=0.002, betas= (0.5, 0.999))#, dampening=0, weight_decay=0.0001)

test_losses = []
test_accuracies = []

best_model_wts = copy.deepcopy(classifier.state_dict())
best_acc = 0.0

#labelled data
labeled_loader = DataLoader(train_set, batch_size=len(train_set), shuffle=False, num_workers=1)
val_loader = DataLoader(val_set, batch_size=len(val_set), shuffle=False, num_workers=1)

labeled_batch = next(iter(labeled_loader))
labeled_data = labeled_batch[0].to(device)
labels = labeled_batch[1].to(device)
val_losses = []
val_accuracies = []

for epoch in range(1, epochs + 1):
    print(f'epoch {epoch}')
    for batch_idx, unlabeledBatch in enumerate(unlabeledDataLoader, 0):
    #for unlabeledBatch in unlabeledDataLoader:
        #print(f"processing {unlabeledBatch.shape}")
       # TRAIN THE DISCRIMINATOR (THE CLASSIFIER)
        classifier.train()
        optimizerC.zero_grad()
        
        # 1. on Unlabelled data
        data = unlabeledBatch.to(device)        
        outputs = classifier(data)    
        logz_unlabel = log_sum_exp(outputs)
        lossUL = 0.5 * (-torch.mean(logz_unlabel) + torch.mean(F.softplus(logz_unlabel)))
        lossUL.backward()  
        
        # 2. on the generated data

        noise = torch.randn(64, rand_input, 1, 1, device=device)
        generated = (netG(noise)+1.0)/2.0
        outputs = classifier(generated.detach()) # detach() because we are not training G here
        logz_fake = log_sum_exp(outputs)
        lossD = 0.5*torch.mean(F.softplus(logz_fake))
        lossD.backward()
        
        # 3. on labeled data
        output = classifier(labeled_data).squeeze()
        logz_label = log_sum_exp(output)
        #print(output.shape)
        #print(labels.unsqueeze(1).shape)
        prob_label = torch.gather(output, 1, labels.unsqueeze(1))
        labeled_loss = -torch.mean(prob_label) + torch.mean(logz_label)
        labeled_loss.backward()    

        optimizerC.step()
        
        # TRAIN THE DISCRIMINATOR (THE CLASSIFIER)
        netG.train()
        optimizerG.zero_grad()
        
        outputs = classifier(generated)
        logz_unlabel = log_sum_exp(outputs)
        lossG = 0.5 * (-torch.mean(logz_unlabel) + torch.mean(F.softplus(logz_unlabel)))
        lossG.backward()
        optimizerG.step()
            
    #unlabeledDataLoader.reset()
    
    generated = (netG(fixed_noise)+1.0)/2.0
    vutils.save_image(generated.cpu().detach(), ('generated_%d.jpg' % epoch), normalize=True)
    
    val_loss, val_accuracy = test(classifier, device, val_loader, True)
    val_losses.append(val_loss)
    val_accuracies.append(val_accuracy)
        
    if val_accuracy > best_acc:
        best_acc = val_accuracy
        best_classifier_wts = copy.deepcopy(classifier.state_dict())


epoch 1
Test set accuracy:  58.333333333333336
epoch 2
Test set accuracy:  63.333333333333336
epoch 3
Test set accuracy:  61.666666666666664
epoch 4
Test set accuracy:  63.333333333333336
epoch 5
Test set accuracy:  58.333333333333336
epoch 6
Test set accuracy:  61.666666666666664
epoch 7
Test set accuracy:  63.333333333333336
epoch 8
Test set accuracy:  41.666666666666664
epoch 9
Test set accuracy:  60.0
epoch 10
Test set accuracy:  55.0
epoch 11
Test set accuracy:  61.666666666666664
epoch 12
Test set accuracy:  63.333333333333336
epoch 13
Test set accuracy:  56.666666666666664
epoch 14
Test set accuracy:  66.66666666666667
epoch 15
Test set accuracy:  66.66666666666667
epoch 16
Test set accuracy:  53.333333333333336
epoch 17
Test set accuracy:  55.0
epoch 18
Test set accuracy:  50.0
epoch 19
Test set accuracy:  51.666666666666664
epoch 20
Test set accuracy:  46.666666666666664
epoch 21
Test set accuracy:  63.333333333333336
epoch 22
Test set accuracy:  66.66666666666667
epoch 23
Tes